In [8]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation

import time
import random

## Environments (Planet)

In [9]:
# U D L R
ACTIONS = {0: (-1, 0), 1: (1, 0), 2: (0, -1), 3: (0, 1)}
class Planet(object):
    def __init__(self):
        # start with defining your planet       
        self.planet = np.zeros((6, 6)).astype(int)
        #Robots
        self.planet[0, 0] = 2
        self.planet[0, 5] = 2
        self.planet[5, 0] = 2
        #Traps
        self.planet[3, 4] = 1
        self.planet[2, 1] = 1
        #self.planet[4, 2] = 1
        #self.planet[0, 3] = 1
        # Robot locations
        self.robot_positions = [(0,0), (0,5), (5,0)]
        self.steps = 0 # contains num steps robots took
        self.allowed_states = None # for now, this is none
        self.construct_allowed_states() 
        
    def is_allowed_move(self, state, action):
        y, x = state
        y += ACTIONS[action][0]
        x += ACTIONS[action][1]
        # moving off the board
        if y < 0 or x < 0 or y > 5 or x > 5:
             return False
        # moving into empty space
        elif self.planet[y, x] != 0:
            return False
        else:
            return True
        
    def construct_allowed_states(self):
        allowed_states = {}
        for y, row in enumerate(self.planet):
            for x, col in enumerate(row):
                # iterate through all valid spaces
                if self.planet[(y,x)] != 1:
                    allowed_states[(y,x)] = []
                    for action in ACTIONS:
                        if self.is_allowed_move((y, x), action):
                            allowed_states[(y,x)].append(action)
        self.allowed_states = allowed_states
        
    def update_planet(self, action, robot_num):
        y, x = self.robot_positions[robot_num]
        old_y, old_x = self.robot_positions[robot_num]
        y += ACTIONS[action][0]
        x += ACTIONS[action][1]
        self.planet[y, x] = 2
        self.planet[old_y, old_x] = 0
        self.robot_positions[robot_num] = (y, x)
        
    def invalid_move(self, action, robot_num):
        y, x = self.robot_positions[robot_num]
        y += ACTIONS[action][0]
        x += ACTIONS[action][1]
        return (y, x)  
                
    def is_game_over(fully_covered):
        return fully_covered
    
    def give_reward(self, state_history, robot_num, visited_locations):
        location = self.robot_positions[robot_num]
        if location not in visited_locations:
            return 1
        else:
            return -0.05
    def get_state_and_reward(self, state_history, robot_num, visited_locations):
        return self.robot_positions[robot_num], self.give_reward(state_history, robot_num, visited_locations)

## Agent Class (Robots)

In [10]:
# U D L R
ACTIONS = {0: (-1, 0), 1: (1, 0), 2: (0, -1), 3: (0, 1)}
class Agent(object):
    def __init__(self, states, alpha=0.15, gamma = 0.9, random_factor=0.2, robot_num=9):
        self.state_history = [((0, 0), 0, 0, (0, 0))] # state, reward, action, next_state
        self.initial_alpha = alpha
        self.alpha = alpha
        self.learning_decay = 1
        self.gamma = gamma
        self.random_factor = random_factor
        self.robot_num = robot_num
        
        # start the rewards table
        self.qtable = np.zeros((36, 4)).tolist()
        
        self.seen_locations = []

    def update_state_history(self, state, reward, action, next_state):
        self.state_history.append((state, reward, action, next_state))
        
    def learn(self, state, reward, action, next_state, episode):
        a = self.alpha
        g = self.gamma
        y, x = state
        state_index = (y * 6) + x
        current_qReward =  self.qtable[state_index][action]
        Ny, Nx = next_state       
        next_state_index = (Ny * 6) + Nx
        self.qtable[state_index][action] = current_qReward + (a * (reward + (g * (max(self.qtable[next_state_index]))) - current_qReward))
        self.state_history = [] # reset the state_history
        self.random_factor = self.random_factor*0.9995 # decrease random_factor
        if self.random_factor <= 0.01:
            self.random_factor = 0.01 
        self.alpha = (1/(1+self.learning_decay*episode))*self.initial_alpha
            
    def choose_action(self, state, allowed_moves, state_history, planet, visited_locations):        
        next_move = None
        n = np.random.uniform()
        # Random
        if n < self.random_factor:
            next_move = np.random.choice(allowed_moves)
            valid = True
        # Choose best action
        else:
            y, x = state
            state_index = (y * 6) + x
            
            unvisited_actions = []
            
            # Compile actions that vist unknown locations
            action = 0
            while action < 4:
                temp_y, temp_x = state
                temp_y += ACTIONS[action][0]
                temp_x += ACTIONS[action][1]
                if not (temp_y < 0 or temp_x < 0 or temp_y > 5 or temp_x > 5):
                    if planet.planet[(temp_y,temp_x)] == 0: 
                        if not ((temp_y, temp_x) in visited_locations):
                            action_reward = self.qtable[state_index][action]
                            unvisited_actions.append((action_reward, action))
                action += 1
            
            #If no unvisited locations looks at all possibilites
            if unvisited_actions == []:
                # Compile all possible actions
                action = 0
                while action < 4:
                    temp_y, temp_x = state
                    temp_y += ACTIONS[action][0]
                    temp_x += ACTIONS[action][1]
                    if not (temp_y < 0 or temp_x < 0 or temp_y > 5 or temp_x > 5):
                        if planet.planet[(temp_y,temp_x)] == 0: 
                            action_reward = self.qtable[state_index][action]
                            unvisited_actions.append((action_reward, action))
                    action += 1                   
                # If still no possible moves
                if unvisited_actions == []:
                    valid = False
                else:
                    action_label = unvisited_actions[unvisited_actions.index(max(unvisited_actions))]
                    unvisited_actions.remove(action_label)
                    next_move = action_label[1]    
                    valid = True                       
            else:
                action_label = unvisited_actions[unvisited_actions.index(max(unvisited_actions))]
                unvisited_actions.remove(action_label)
                next_move = action_label[1]    
                valid = True
        return next_move, valid           

## Server

In [11]:
class Server(object):
    def init(self):
        # Global Variables
        self.fully_covered = False
        
        # Track visited locations
        self.no_visited_locations = 3
        self.visited_locations = [(0,0), (0,5), (5,0)]
        
    def reset_count(self):
        # Reinitialise varaibles
        self.fully_covered = False
        self.no_visited_locations = 3
        self.visited_locations = [(0,0), (0,5), (5,0)]

## Learning Loop

In [ ]:
if __name__ == '__main__':  
    #Hyper Parameters
    Learning_Rate = 0.001
    Error_Rate = 0.9
    Epsilon = 0.5
    Episodes = 5000
    Movement_Budget = 500
    completed_count = 0
    
    planet = Planet()
    server = Server()
    robot_0 = Agent(planet.planet, alpha=Learning_Rate, gamma=Error_Rate, random_factor=Epsilon, robot_num=0)
    robot_1 = Agent(planet.planet, alpha=Learning_Rate, gamma=Error_Rate, random_factor=Epsilon, robot_num=1)
    robot_2 = Agent(planet.planet, alpha=Learning_Rate, gamma=Error_Rate, random_factor=Epsilon, robot_num=2)
    swarm = [robot_0, robot_1, robot_2]
    moveHistory = []
    epsilonHistory = []
    
    # Possible visitable locations
    visitable_locations = (np.size(planet.planet) - np.count_nonzero(planet.planet == 1))
    
    for i in range(Episodes):
        if i % 1000 == 0:                    
            print("Episode",i)
            print("Completed", completed_count) 
            completed_count = 0
        movementBudget = Movement_Budget
        server.reset_count()
        
        while not server.fully_covered:
            n = 0
            while n < 3:
                robot = swarm[n]
                sHistory = robot.state_history
                state, _ = planet.get_state_and_reward(sHistory, robot.robot_num, server.visited_locations) # get the current state
                action, valid = robot.choose_action(state, planet.allowed_states[state], sHistory, planet, server.visited_locations) # choose an action (explore or exploit)
                # There is a possible move for the robot
                if valid:
                    planet.update_planet(action, robot.robot_num) # update the planet according to the action
                    next_state, reward = planet.get_state_and_reward(sHistory, robot.robot_num, server.visited_locations) # get the new state and reward    
                    if next_state not in server.visited_locations:
                        server.visited_locations.append(next_state)
                        server.no_visited_locations += 1
                        if server.no_visited_locations >= visitable_locations:
                            completed_count += 1
                            reward = 10
                    robot.learn(state, reward, action, next_state, i)
                n += 1
                
            planet.steps += 1
            movementBudget -= 1
            if server.no_visited_locations >= visitable_locations: 
                server.fully_covered = True
            elif movementBudget <= 0:
                # End the episode if the movement budget runs out
                server.fully_covered = True        
        
        # robot should learn after every episode

        moveHistory.append(planet.steps) # get a history of number of steps taken to plot later
        epsilonHistory.append(robot.random_factor)
        planet = Planet() # reinitialize the planet

print("Episode",i)
print("Epsilon",robot.random_factor)
print("Completed", completed_count) 
        
print("Visited Locations:", server.no_visited_locations, "Steps:", moveHistory[-1])  
plt.figure(figsize=(18, 10))
plt.semilogy(moveHistory, color="b")
#plt.semilogy(epsilonHistory, color="r")
plt.show()

## Testing and Simulation

In [ ]:
testHistory = []

server.reset_count()

planet = Planet()
tx, ty = [], []
p0x, p0y, r0x, r0y, x_history_0, y_history_0 = [], [], [], [], [], []
p1x, p1y, r1x, r1y, x_history_1, y_history_1 = [], [], [], [], [], []
p2x, p2y, r2x, r2y, x_history_2, y_history_2 = [], [], [], [], [], []

for y, row in enumerate(planet.planet):
    for x, col in enumerate(row):
        if planet.planet[(y,x)] == 1:
            tx.append(x)
            ty.append(y)

r0y.append(planet.robot_positions[0][0])
r0x.append(planet.robot_positions[0][1])
r1y.append(planet.robot_positions[1][0])
r1x.append(planet.robot_positions[1][1])
r2y.append(planet.robot_positions[2][0])
r2x.append(planet.robot_positions[2][1])

plt.scatter(r0x, r0y, marker = 's', color = 'g') 
plt.scatter(r1x, r1y, marker = 's', color = 'b') 
plt.scatter(r2x, r2y, marker = 's', color = 'y') 
plt.scatter(tx, ty, marker = 'x', color = 'r') 
plt.xlim([-1, 6])
plt.ylim([-1, 6])   
plt.show()

movementBudget = Movement_Budget
server.reset_count()
robot.random_factor = 0
vist_count = 0
action_history = []

while not server.fully_covered:
    n = 0
    while n < 3:
        robot = swarm[n]
        sHistory = robot.state_history
        state, _ = planet.get_state_and_reward(sHistory, robot.robot_num, server.visited_locations) # get the current state
        action, valid = robot.choose_action(state, planet.allowed_states[state], sHistory, planet, server.visited_locations) # choose an action (explore or exploit)
        # There is a possible move for the robot
        if valid:
            planet.update_planet(action, robot.robot_num) # update the planet according to the action
            next_state, reward = planet.get_state_and_reward(sHistory, robot.robot_num, server.visited_locations) # get the new state and reward    
            if next_state not in server.visited_locations:
                server.visited_locations.append(next_state)
                server.no_visited_locations += 1
                if server.no_visited_locations >= visitable_locations:
                    completed_count += 1
                    reward = 10
            robot.learn(state, reward, action, next_state, i)
        n += 1

    planet.steps += 1
    movementBudget -= 1
    if server.no_visited_locations >= visitable_locations: 
        server.fully_covered = True
    elif movementBudget <= 0:
        # End the episode if the movement budget runs out
        server.fully_covered = True  

    # Get locations of robots for plotting
    r0y.append(planet.robot_positions[0][0])
    r0x.append(planet.robot_positions[0][1])
    r1y.append(planet.robot_positions[1][0])
    r1x.append(planet.robot_positions[1][1])
    r2y.append(planet.robot_positions[2][0])
    r2x.append(planet.robot_positions[2][1])
    
r0y.append(planet.robot_positions[0][0])
r0x.append(planet.robot_positions[0][1])
r1y.append(planet.robot_positions[1][0])
r1x.append(planet.robot_positions[1][1])
r2y.append(planet.robot_positions[2][0])
r2x.append(planet.robot_positions[2][1])    
    
testHistory.append(planet.steps) # get a history of number of steps taken to plot later

fig, ax = plt.subplots()

def animate(i):
    print(i)
    plt.clf()
    plt.xlim([-1, 6])
    plt.ylim([-1, 6])
    plt.scatter(tx, ty, marker = 'x', color = 'r') 
    plt.scatter(r0x[i], r0y[i], marker = 's', color = 'g') 
    plt.scatter(r1x[i], r1y[i], marker = 's', color = 'b') 
    plt.scatter(r2x[i], r2y[i], marker = 's', color = 'y')
    p0x.append(r0x[i])
    p0y.append(r0y[i])
    p1x.append(r1x[i])
    p1y.append(r1y[i])
    p2x.append(r2x[i])
    p2y.append(r2y[i])
    plt.scatter(p0x, p0y, marker = '.', color = 'g')
    plt.scatter(p1x, p1y, marker = '.', color = 'b')
    plt.scatter(p2x, p2y, marker = '.', color = 'y')        
    return fig

ani = animation.FuncAnimation(fig, animate, repeat=True,
                                    frames=len(r0x) - 1, interval=500) 

# To save the animation using Pillow as a gif
writer = animation.PillowWriter(fps=3,
                             metadata=dict(artist='Me'),
                             bitrate=1800)
ani.save('path.gif', writer=writer)
plt.show()
        